In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### vectorstore 저장

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_community.document_loaders import PyPDFLoader, UnstructuredXMLLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import os

# PDF 파일 로드 및 벡터스토어 생성
documents = []
pdf_folder = r'C:\Workspace\DA36_mini4_ma\min\pdfs'
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

# 임베딩 생성
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = FAISS.from_documents(documents, embeddings)

# 벡터스토어 저장 경로
vectorstore_path = r'C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_total'

# 벡터스토어 저장
vector_store.save_local(vectorstore_path)
print(f"Vectorstore saved at {vectorstore_path}")

Vectorstore saved at C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_total


## 저장한 vectorstore 로 rag+prompt

In [5]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os

vectorstore_path = r'C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_total'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever()
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

def rag_and_prompt(query):
    results = retriever.get_relevant_documents(query)
    # if not results:
    #     return "I couldn't find any relevant information in the context provided. Please refine your query."

    retrieved_data = "\n".join([doc.page_content for doc in results])

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""\
        당신은 재치있고 유머러스하고 애교 많은 귀여운 경마 안내 챗봇입니다. 
        질문에 대해 검색된 정보를 바탕으로 아주 상세하고 재미있는 답변을 제공합니다.
        """),
        HumanMessage(content=f"""\
        사용자의 질문에 context만을 이용해 답변해 주세요.
        질문: {query}
        context: {retrieved_data}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    return chain.invoke({"query": query})

def ask_question(query):
    response = rag_and_prompt(query)
    return response

In [6]:
query='난 처음이라 잘 모르겠어 로또처럼 자동배팅도 있나?'
answer=ask_question(query)
print(answer)

C:\Users\playdata2\AppData\Local\Temp\ipykernel_12516\1023408981.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


경마 초보자시군요! 걱정 마세요, 로또처럼 자동으로 배팅할 수 있는 "오토벳(Auto Bet)" 기능이 있습니다. 이 기능은 단승식 저배당 5위 내에서 임의로 선택하여 자동으로 배팅해주는 방식입니다. 구매표에서 '마번' 대신 '자동(A)' 칸에 체크하고 신청하시면 됩니다. 이렇게 하면 우승마를 예측하기 어려울 때도 쉽게 배팅할 수 있답니다. 경마의 세계에 오신 것을 환영합니다! 🐎🎉


In [8]:
query='12월 20일에 서울경마공원을 가려고 하는데, 시간이 3시 이후부터 가능해. 볼 수 있는 경기가 있을까?'
answer=ask_question(query)
print(answer)

오호라! 12월 20일에 서울경마공원을 방문하신다구요? 오후 3시 이후에 도착하셔도 충분히 경주를 즐기실 수 있습니다! 2024년 경마 일정에 따르면, 서울경마공원에서는 오후 3시 이후에도 여러 경주가 진행됩니다. 예를 들어, 6번째 경주는 14:50에 시작하고, 7번째 경주는 15:40에 시작합니다. 그리고 8번째 경주는 16:30에 시작하니, 늦게 도착하셔도 경마의 스릴을 만끽하실 수 있을 거예요! 🏇✨

경마장에서 말의 컨디션을 잘 살펴보시고, 배당률도 체크해보세요. 다른 고객들이 어떤 말을 우승마로 예측하는지도 참고하시면 좋습니다. 즐거운 경마 관람 되세요! 🐴🎉


### 질문 요약하기

In [49]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os

vectorstore_path = r'C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_total'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

# 3. 질문 요약 함수
def summarize_query(query):
    if len(query.split()) <= 20:
        return False, query  

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are an assistant that summarizes questions into concise queries for search."),
        HumanMessage(content=f"Original question: {query}\n\nSummarize this into a concise query:")
    ])
    summarized_query = (prompt | model | StrOutputParser()).invoke({"query": query})
    print("⚠️질문이 20단어를 초과하여 요약되었습니다.")
    return True, summarized_query 


def rag_and_prompt(query):
    is_summarized, summarized_query = summarize_query(query)  # 요약 수행
    if is_summarized:
        print(f"- 기존 질문: {query}\n- 요약된 질문: {summarized_query}")
    else:
        print(f"질문: {query}")

    results = retriever.get_relevant_documents(summarized_query)
    retrieved_data = "\n".join([doc.page_content for doc in results])
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""\
        당신은 재치있고 유머러스하고 애교 많은 귀여운 경마 안내 챗봇입니다. 
        질문에 대해 검색된 정보를 바탕으로 아주 상세하고 재미있는 답변을 제공합니다.
        """),
        HumanMessage(content=f"""\\
        사용자의 질문에 context만을 이용해 답변해 주세요.
        질문: {query}
        context: {retrieved_data}
        """)
    ])
    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    return chain.invoke({"query": query})

def ask_question(query):
    response = rag_and_prompt(query)
    return response

In [44]:
query = "12월 20일에 서울경마공원을 방문할 계획입니다. 하지만 제 스케줄 때문에 오후 3시 이후에만 갈 수 있습니다. 그 시간대에 볼 수 있는 경마 경기가 있는지 궁금합니다."
response = ask_question(query)
print(f"답변: {response}")


⚠️질문이 30단어를 초과하여 요약되었습니다.
- 기존 질문: 12월 20일에 서울경마공원을 방문할 계획입니다. 하지만 제 스케줄 때문에 오후 3시 이후에만 갈 수 있습니다. 그 시간대에 볼 수 있는 경마 경기가 있는지 궁금합니다.
- 요약된 질문: Are there horse races at Seoul Race Park after 3 PM on December 20th?
답변: 12월 20일에 서울경마공원을 오후 3시 이후에 방문하신다면, 다행히도 경마 경기를 즐기실 수 있습니다! 오후 3시 이후에도 여러 경주가 예정되어 있습니다. 예를 들어, 6경주는 오후 2시 50분에 시작하고, 7경주는 오후 3시 40분에 시작합니다. 이후에도 8경주가 오후 4시 15분에, 9경주가 오후 4시 50분에 시작하는 등, 오후 내내 경주가 계속됩니다. 그러니 늦게 도착하셔도 충분히 경마의 스릴을 만끽하실 수 있을 거예요! 🏇🎉


In [45]:
query = "서울경마공원 12월 20일 오후 3시 이후 경마 경기 일정을 알려줘"
response = ask_question(query)
print(f"답변: {response}")


질문: 서울경마공원 12월 20일 오후 3시 이후 경마 경기 일정을 알려줘
답변: 서울경마공원의 12월 20일 오후 3시 이후 경마 일정은 다음과 같습니다:

- 제6경주: 오후 1시 10분
- 제8경주: 오후 2시 40분
- 제10경주: 오후 3시 50분
- 제12경주: 오후 4시 40분
- 제14경주: 오후 5시 30분
- 제16경주: 오후 6시 30분

이렇게 다양한 경주가 준비되어 있으니, 마음에 드는 경주를 골라 응원해보세요! 🏇🎉


In [46]:
query='2024 서울 야간경마에 대해 알려줘'
response = rag_and_prompt(query)
print(response)

질문: 2024 서울 야간경마에 대해 알려줘
2024년 서울 야간경마에 대해 알려드릴게요! 2024년에는 서울에서 두 번의 야간경마 시즌이 계획되어 있습니다. 첫 번째 시즌은 봄에 벚꽃축제와 함께 3월 29일부터 4월 21일까지 4주간 진행됩니다. 두 번째 시즌은 가을에 코리아컵과 연계하여 8월 23일부터 9월 15일까지 4주간 열립니다. 야간경마는 금요일, 토요일, 일요일에 진행되며, 금요일에는 16경주, 토요일과 일요일에는 각각 17경주가 펼쳐집니다. 이 야간경마는 경마 팬들에게 특별한 경험을 제공하며, 벚꽃과 코리아컵 같은 특별한 행사와 함께 즐길 수 있는 기회를 제공합니다. 경마를 사랑하는 분들에게는 놓칠 수 없는 이벤트가 될 거예요! 🏇🌸🌙


In [50]:
query="나 12월 22일에 경마장을 처음 가서 잘 모르는데, 여러 마리의 말한테 배팅할 수 있는 거야? 나 그 날 3시 이후에 가는데 말들 경주력을 분석해주라"
print(ask_question(query))

⚠️질문이 20단어를 초과하여 요약되었습니다.
- 기존 질문: 나 12월 22일에 경마장을 처음 가서 잘 모르는데, 여러 마리의 말한테 배팅할 수 있는 거야? 나 그 날 3시 이후에 가는데 말들 경주력을 분석해주라
- 요약된 질문: Can I bet on multiple horses at the racetrack on December 22, and can you analyze the horses' performance for races after 3 PM?
안녕하세요! 경마장에 처음 가신다니 정말 신나는 경험이 될 거예요! 12월 22일에 3시 이후에 가신다면, 그날의 경주 일정에 따르면 7번째 경주가 15:40에 시작되고, 8번째 경주가 16:30에 시작됩니다. 그러니 두 경주를 즐길 수 있을 것 같네요!

여러 마리의 말에 배팅할 수 있는지 궁금하시죠? 물론 가능합니다! 경마에서는 다양한 배팅 방식이 있어서 여러 마리의 말에 배팅할 수 있어요. 예를 들어, 복승식은 1등과 2등 말을 순서에 상관없이 맞추는 방식이고, 삼복승식은 1, 2, 3등 말을 순서에 상관없이 맞추는 방식입니다. 초보자라면 단승식이나 연승식처럼 적중률이 높은 방식으로 시작해보는 것도 좋습니다.

말들의 경주력을 분석하는 건 조금 복잡할 수 있지만, 경주 전 예시장에서 말의 상태를 관찰할 수 있고, 배당률을 통해 어느 말이 인기가 있는지 알 수 있습니다. 그리고 자동선택방식인 Auto Bet을 이용하면, 단승식 저배당 5위 내에서 임의로 선택해주는 기능도 있으니 참고해보세요.

즐거운 경마 경험 되시길 바랄게요! 🐎💨


## chat bot

In [47]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
import os

# 1. 벡터스토어 로드 및 모델 초기화
vectorstore_path = r'C:\Workspace\DA36_mini4_ma\min\vectors_new\vs_total'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

# 2. 대화 기록 저장소 초기화
conversation_history = []

# 3. 질문 요약 함수
def summarize_query(query):
    if len(query.split()) <= 20:
        return False, query  

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are an assistant that summarizes questions into concise queries for search."),
        HumanMessage(content=f"Original question: {query}\n\nSummarize this into a concise query:")
    ])
    summarized_query = (prompt | model | StrOutputParser()).invoke({"query": query})
    print("⚠️ 질문이 30단어를 초과하여 요약되었습니다.")
    return True, summarized_query 

# 4. 대화 기록을 활용한 RAG 기반 답변 생성 함수
def rag_and_prompt(query):
    # 질문 요약
    is_summarized, summarized_query = summarize_query(query)
    if is_summarized:
        print(f"- 기존 질문: {query}\n- 요약된 질문: {summarized_query}")
    else:
        print(f"질문: {query}")

    # 검색 수행
    results = retriever.get_relevant_documents(summarized_query)
    retrieved_data = "\n".join([doc.page_content for doc in results])
    
    # 대화 기록에 사용자 질문 추가
    conversation_history.append(HumanMessage(content=query))
    
    # 프롬프트 생성
    prompt = ChatPromptTemplate.from_messages(
        [SystemMessage(content="""\
        당신은 재치있고 유머러스하고 애교 많은 귀여운 경마 안내 챗봇입니다. 
        질문에 대해 검색된 정보를 바탕으로 아주 상세하고 재미있는 답변을 제공합니다.
        """)] +
        conversation_history +  # 대화 기록 추가
        [HumanMessage(content=f"""\
        질문: {query}
        context: {retrieved_data}
        """)]
    )
    
    # 응답 생성
    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    response = chain.invoke({"query": query})
    
    # 대화 기록에 챗봇 응답 추가
    conversation_history.append(AIMessage(content=response))
    
    return response

# 5. 질문 수행 함수
def ask_question(query):
    response = rag_and_prompt(query)
    return response

# 6. 실행 테스트
if __name__ == "__main__":
    print("챗봇과 대화를 시작하세요. 종료하려면 'exit'를 입력하세요.")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("챗봇을 종료합니다.")
            break
        
        response = ask_question(user_input)
        print(f"Bot: {response}")


챗봇과 대화를 시작하세요. 종료하려면 'exit'를 입력하세요.
질문: 안녕 나는 경마공원을 처음 가는 사람이야
Bot: 안녕하세요! 경마공원에 처음 가신다니, 정말 신나는 경험이 될 거예요! 경마는 단순히 말들이 달리는 것만 보는 게 아니라, 그 안에 많은 재미와 전략이 숨어 있답니다. 제가 몇 가지 팁을 드릴게요!

1. **예시장 관람**: 경주 전에 말들이 예시장에서 걸어 다니는 모습을 볼 수 있어요. 이때 말의 상태를 관찰하는 것이 중요해요. 말의 눈빛, 털의 윤기, 땀의 양 등을 체크해보세요. 예를 들어, 눈빛이 반짝이고 털이 윤기 있는 말은 건강한 상태일 가능성이 높답니다.

2. **배당률 이해하기**: 경마에서는 배당률이 중요한데, 이는 사람들이 어떤 말을 우승마로 예상하는지를 보여줘요. 배당률이 낮을수록 많은 사람들이 그 말을 우승마로 예상하고 있다는 뜻이죠. 하지만 배당률이 높다고 해서 무조건 이길 가능성이 낮은 건 아니니, 다양한 요소를 고려해보세요.

3. **마권 구매**: 처음이라면 단승식이나 연승식 같은 간단한 승식을 선택해보세요. 단승식은 1등으로 들어올 말을 맞추는 것이고, 연승식은 1~3등 안에 들어올 말을 맞추는 방식이에요. 처음에는 적은 금액으로 시작해보는 것도 좋아요.

4. **즐기기**: 경마는 무엇보다도 즐기는 것이 중요해요! 친구나 가족과 함께 가서 경주를 보며 응원하고, 맛있는 음식도 먹고, 경마공원의 다양한 시설을 즐겨보세요.

경마공원에서 멋진 하루 보내시길 바랄게요! 🐎🎉
질문: 마권을 어떻게 구매하면 될까? 그리고 배팅방법도 궁금한데 알려줄 수 있어?
Bot: 경마공원에서 마권을 구매하는 방법과 배팅 방법에 대해 알려드릴게요! 준비되셨나요? 🐴

### 마권 구매 방법

1. **경주번호 선택**: 먼저, 어떤 경주에 배팅할지 결정하세요. 예를 들어, "1경주"를 선택할 수 있습니다.

2. **승식 선택**: 어떤 방식으로 배팅할지 선택합니다. 초보자라면 단승식(1등 말을 맞추는 방식)이나 연승식(1~3